# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.  

Требуется построить модель с максимально большим значением accuracy, по крайней мере больше 0.75. 

### План работы:

1. [Знакомство с данными](#1.-Знакомство-с-данными.)
   
2. [Разделение данных на выборки](#2.-Разделение-данных-на-выборки.)

3. [Исследование качества разных моделей](#3.-Исследование-качества-разных-моделей.)

4. [Проверка на тестовой выборке](#4.-Проверка-на-тестовой-выборке.)

5. [Проверка модели на адекватность](#5.-Проверка-модели-на-адекватность.)
    
6. [Общий вывод](#6.-Общий-вывод.)

### Описание данных  
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.   
Известно:  
`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

---

## 1. Знакомство с данными.

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score 

In [52]:
df = pd.read_csv('/datasets/users_behavior.csv') 

In [53]:
display(df.shape)
display(df.head(5))

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [55]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [56]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [57]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

### Вывод
Проверили, что с данными все в порядке. Целевой признак: тариф ультра или нет, значит будем решать задачу бинарной (двоичной) классификации.

## 2. Разделение данных на выборки.

Поделим наши данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [58]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(df.shape)

(1928, 4)
(643, 4)
(643, 4)
(3214, 5)


## 3. Исследование качества разных моделей.

Исследуем 3 модели: решающее дерево, случайный лес и логистическая регрессия.

In [59]:
best_model = None
best_result = 0
for depth in range(1, 6):
    for samples_split in range(2,6):
        for samples_leaf in range(1,6):
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=samples_split, min_samples_leaf=samples_leaf)
            model.fit(features, target) 
            predictions = model.predict(features_valid) 
            result = accuracy_score(target_valid, predictions) 
 
            if result > best_result:
                best_model = model
                best_result = result
print(best_model)
print(best_result)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
0.7978227060653188


In [60]:
best_model = None
best_result = 0
for depth in range(1, 6):
    for est in range(1,11):
        for samples_split in range(2,6):
            for samples_leaf in range(1,6):
                model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_split=samples_split, min_samples_leaf=samples_leaf)
                model.fit(features, target) 
                predictions = model.predict(features_valid) 
                result = accuracy_score(target_valid, predictions) 
 
                if result > best_result:
                     best_model = model
                     best_result = result
print(best_model)
print(best_result)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
0.7931570762052877


In [61]:
best_model = None
best_result = 0
for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model = LogisticRegression(penalty='l2', random_state=12345, solver=solver, max_iter=10000)
    model.fit(features, target) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions) 
    print('penalty', penalty,'solver', solver, 'accuracy: ', result)
 
    if result > best_result:
        best_model = model
        best_result = result
print(best_model)
print(best_result)

penalty l2 solver newton-cg accuracy:  0.7247278382581649
penalty l2 solver lbfgs accuracy:  0.7247278382581649
penalty l2 solver liblinear accuracy:  0.6982892690513219
penalty l2 solver sag accuracy:  0.6998444790046656
penalty l2 solver saga accuracy:  0.7013996889580093
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='newton-cg', tol=0.0001,
                   verbose=0, warm_start=False)
0.7247278382581649


### Вывод
Наилучшее качество модели на валидационной выборке показала модель решающего дерева с глубиной 5, минимальным количеством примеров для разделения 2 и минимальным количеством объектов в листе 1. Максимальное количество итераций в логистической регрессии пришлось увеличить до 10000 из-за проблем со сходимостью оптимизационного алгоритма *sag*.

## 4. Проверка на тестовой выборке.

Проверим наилучшую модель на тестовой выборке:

In [62]:
model = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
model.fit(features, target) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions) 
print(result)

0.80248833592535


### Вывод
Качество модели 0.80, что выше качества (0.75) в задании.

## 5. Проверка модели на адекватность.


Проверим модель на адекватность с помощью DummyClassifier:

In [63]:
model = DummyClassifier(strategy='most_frequent')
model.fit(features, target) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions) 
print(result)

0.6951788491446346


### Вывод
Качество нашей модели выше, чем рандомный ответ, значит проверку на адекватность она прошла. 

## 6. Общий вывод.
Были проверены 3 модели для рекомендации тарифов: решающее дерево, случайный лес и логистическая регрессия. Лучшее качество на валидационной выборке (0.797) показала модель решающего дерева с параметрами `max_depth=5, min_samples_leaf=1, min_samples_split=2`. Эта модель была проверена на тестовой выборке и показала качество 0.802, что очень близко значению для валидационной выборки, и означает, что мы не столкнулись с проблемой переобучения или недообучения. Выбранная модель прошла тест на адекватность, т.е. показала качество лучше, чем рандомный ответ.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75